# Bottle Processing
Author: Andrew Reed

### Motivation:
Independent verification of the suite of physical and chemical observations provided by OOI are critical for the observations to be of use for scientifically valid investigations. Consequently, CTD casts and Niskin water samples are made during deployment and recovery of OOI platforms, vehicles, and instrumentation. The water samples are subsequently analyzed by independent labs for  comparison with the OOI telemetered and recovered data.

However, currently the water sample data routinely collected and analyzed as part of the OOI program are not available in a standardized format which maps the different chemical analyses to the physical measurements taken at bottle closure. Our aim is to make these physical and chemical analyses of collected water samples available to the end-user in a standardized format for easy comprehension and use, while maintaining the source data files. 

### Approach:
Generating a summary of the water sample analyses involves preprocessing and concatenating multiple data sources, and accurately matching samples with each other. To do this, I first preprocess the ctd casts to generate bottle (.btl) files using the SeaBird vendor software following the SOP available on Alfresco. 

Next, the bottle files are parsed using python code and the data renamed following SeaBird's naming guide. This creates a series of individual cast summary (.sum) files. These files are then loaded into pandas dataframes, appended to each other, and exported as a csv file containing all of the bottle data in a single data file.

### Data Sources/Software:

* **sbe_name_map**: This is a spreadsheet which maps the short names generated by the SeaBird SBE DataProcessing Software to the associated full names. The name mapping originates from SeaBird's SBE DataProcessing support documentation.

* **Alfresco**: The Alfresco CMS for OOI at alfresco.oceanobservatories.org is the source of the ctd hex, xmlcon, and psa files necessary for generating the bottle files needed to create the sample summary sheet.

* **SBEDataProcessing-Win32**: SeaBird vendor software for processing the raw ctd files and generating the .btl files.


**========================================================================================================================**
Import packages which will be used in this notebook:

In [1]:
import os, sys, re
import pandas as pd
import numpy as np

Load the name mapping for the column names based on SeaBird's manual:

In [2]:
sbe_name_map = pd.read_excel('/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')

In [3]:
sbe_name_map.head()

,Short Name,Full Name,Friendly Name,Units,Notes/Comments
0,accM,Acceleration [m/s^2],acc M,m/s^2,NaN
1,accF,Acceleration [ft/s^2],acc F,ft/s^2,NaN
2,altM,Altimeter [m],alt M,m,NaN
3,altF,Altimeter [ft],alt F,ft,NaN
4,avgsvCM,"Average Sound Velocity [Chen-Millero, m/s]",avgsv-C M,"Chen-Millero, m/s",NaN


**========================================================================================================================**
Declare the directory paths to where the relevant information is stored:

In [4]:
basepath = '/home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/'
array = 'Pioneer/'
cruise = 'Pioneer-03_Leg-1_KN-222_2014-10-03/'
water = 'Water Sampling/'
ctd = 'ctd/'
leg = ''

In [5]:
sorted(os.listdir(basepath+array+cruise+water))

['.~lock.Pioneer-03_Leg-1_KN-222_Chlorophyll_Sample_Data_2017-09-21_ver_1-00.xlsx#',
 '.~lock.Pioneer-03_Leg-1_KN-222_Sampling_Log.xlsx#',
 'Pioneer-03_Leg-1_KN-222_CTD_Sampling_Log-written.pdf',
 'Pioneer-03_Leg-1_KN-222_Chlorophyll_Sample_Data_2017-09-21_ver_1-00.xlsx',
 'Pioneer-03_Leg-1_KN-222_DIC_Sample_Data_2019-06-19_ver_1-00.xlsx',
 'Pioneer-03_Leg-1_KN-222_Nutrients_Sample_Data_2019-06-24_ver_1-00.xlsx',
 'Pioneer-03_Leg-1_KN-222_Oxygen_Salinity_Sample_Data',
 'Pioneer-03_Leg-1_KN-222_Sampling_Log.xlsx']

In [ ]:
sample_dir = basepath+array+cruise+leg+ctd
water_dir = basepath+array+cruise+water
salts_and_o2_path = water_dir+ 'Pioneer-03_Leg-1_KN-222_Oxygen_Salinity_Sample_Data/'
log_path = water_dir+ 'Pioneer-03_Leg-1_KN-222_Sampling_Log.xlsx'
nutrients_path = water_dir+ 'Pioneer-03_Leg-1_KN-222_Nutrients_Sample_Data_2019-06-24_ver_1-00.xlsx'
chl_path = water_dir+ 'Pioneer-03_Leg-1_KN-222_Chlorophyll_Sample_Data_2017-09-21_ver_1-00.xlsx'
dic_path = water_dir + 'Pioneer-03_Leg-1_KN-222_DIC_Sample_Data_2019-06-19_ver_1-00.xlsx'

In [ ]:
# Parse the data for the start_time
def parse_header(header):
    """
    Parse the header of bottle (.btl) files to get critical information
    for the summary spreadsheet.
    
    Args:
        header - an object containing the header of the bottle file as a list of
            strings, split at the newline.
    Returns:
        hdr - a dictionary object containing the start_time, filename, latitude,
            longitude, and cruise id.
    """
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time [UTC]':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Filename':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude [degrees]':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude [degrees]':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise':cruise_id})
        else:
            pass
    
    return hdr

Get the path to the ctd-bottle data, load it, and parse it:

In [ ]:
# Now write a function to autopopulate the bottle summary sample sheet
files = [x for x in os.listdir(sample_dir) if '.btl' in x]
for filename in files:
    filepath = os.path.abspath(sample_dir+filename)
    
    # Load the raw content into memory
    with open(filepath) as file:
        content = file.readlines()
    content = [x.strip() for x in content]
    
    # Now parse the file content
    header = []
    columns = []
    data = []
    for line in content:
        if line.startswith('*') or line.startswith('#'):
            header.append(line)
        else:
            try:
                float(line[0])
                data.append(line)
            except:
                columns.append(line)
                
    # Parse the header
    hdr = parse_header(header)
    
    # Parse the column identifiers
    column_dict = {}
    for line in columns:
        for i,x in enumerate(line.split()):
            try:
                column_dict[i] = column_dict[i] + ' ' + x
            except:
                column_dict.update({i:x})
                
    #Parse the bottle data based on the column header locations
    data_dict = {x:[] for x in column_dict.keys()}

    for line in data:
        if line.endswith('(avg)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            for i,x in enumerate(values):
                data_dict[i].append(x)
        elif line.endswith('(sdev)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            data_dict[1].append(values[0])
        else:
            pass
    
    # Join the date and time for each measurement into a single item
    data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]
    
    # With the parsed data and column names, match up the data and column
    # based on the location
    results = {}
    for key,item in column_dict.items():
        values = data_dict[key]
        results.update({item:values})
        
    # Put the results into a dataframe
    df = pd.DataFrame.from_dict(results)

    # Now add the parsed info from the header files into the dataframe
    for key,item in hdr.items():
        df[key] = item
        
    # Get the cast number
    cast = filename[filename.index('.')-3:filename.index('.')]
    df['Cast'] = str(cast).zfill(3)
    
    # Add the header info back in
    for key in hdr.keys():
        df[key] = hdr[key]
        
    # Generate a filename for the summary file
    outname = filename.split('.')[0] + '.sum'
    
    # Save the results
    df.to_csv(sample_dir+outname)

In [ ]:
# Now, for each "summary" file, load and append to each other
df = pd.DataFrame()
for file in os.listdir(sample_dir):
    if '.sum' in file:
        df = df.append(pd.read_csv(sample_dir+file))
    else:
        pass

In [ ]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [ ]:
df

In [ ]:
df.sort_values(by=['Cast','Bottle Position'], inplace=True)
df.drop(columns='Unnamed: 0',inplace=True)
for colname in list(df.columns.values):
    df.rename({colname:'CTD ' + colname},axis='columns',inplace=True)
bottles = df

In [ ]:
df.to_csv(sample_dir+'CTD_Summary.csv')

In [ ]:
df.head()

**========================================================================================================================**
### Process the Discrete Salinity and Oxygen Data
Next, I process the discrete salinity and oxygen sample data so that it is consistently named and ready to be merged with the existing data sets.

In [ ]:
def clean_sal_files(dirpath):

    # Run check if files are held in excel format or csvs
    csv_flag = any(files.endswith('.SAL') for files in os.listdir(dirpath))
    if csv_flag:
        for filename in os.listdir(dirpath):
            sample = []
            salinity = []
            if filename.endswith('.SAL'):
                with open(dirpath+filename) as file:
                    data = file.readlines()
                    for ind1,line in enumerate(data):
                        if ind1 == 0:
                            strs = data[0].replace('"','').split(',')
                            cruisename = strs[0]
                            station = strs[1]
                            cast = strs[2]
                            case = strs[8]
                        elif int(line.split()[0]) == 0:
                            pass
                        else:
                            strs = line.split()
                            sample.append(strs[0])
                            salinity.append(strs[2])
                
                    # Generate a pandas dataframe to populate data
                    data_dict = {'Cruise':cruisename,'Station':station,'Cast':cast,'Case':case,'Sample ID':sample,'Salinity [psu]':salinity}
                    df = pd.DataFrame.from_dict(data_dict)
                    df.to_csv(file.name.replace('.','')+'.csv')
            else:
                pass
    
    else:
        # If the files are already in excel spreadsheets, they've been cleaned into a
        # logical tabular format
        pass
    

def process_sal_files(dirpath):
    
    # Check if the files are excel files or not
    excel_flag = any(files.endswith('SAL.xlsx') for files in os.listdir(dirpath))
    # Initialize a dataframe for processing the salinity files
    df = pd.DataFrame()
    if excel_flag:
        for file in os.listdir(dirpath):
            if 'SAL.xlsx' in file:
                df = df.append(pd.read_excel(dirpath+file))
        df.rename({'Sample':'Sample ID','Salinity':'Salinity [psu]','Niskin #':'Niskin','Case ID':'Case'}, 
                  axis='columns',inplace=True)
        df.dropna(inplace=True)
        df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
        df['Niskin'] = df['Niskin'].apply(lambda x: str( int(x)))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    else:
        for file in os.listdir(dirpath):
            if 'SAL.csv' in file:
                df = df.append(pd.read_csv(dirpath+file))
        df.dropna(inplace=True)
        df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
        df.drop(columns=[x for x in list(df.columns.values) if 'unnamed' in x.lower()],inplace=True)

    # Save the processed summary file for salinity
    df.to_csv(dirpath+'SAL_Summary.csv')
    
    
def process_oxy_files(dirpath):
    df = pd.DataFrame()
    for filename in os.listdir(dirpath):
        if 'oxy' in filename.lower() and filename.endswith('.xlsx'):
            df = df.append(pd.read_excel(dirpath+filename)) 
            # Rename and clean up the oxygen data to be uniform across data sets
    df.rename({'Niskin #':'Niskin','Sample#':'Sample ID','Oxy':'Oxygen [mL/L]','Unit':'Units'},
              axis='columns',inplace=True)
    df.dropna(inplace=True)
    df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
    #df['Niskin'] = df['Niskin'].apply(lambda x: str( int(x)))
    df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    df['Cruise'] = df['Cruise'].apply(lambda x: x.replace('O','0'))
    
    # Save the processed summary file for oxygen
    df.to_csv(dirpath+'OXY_Summary.csv')

**If there is no oxygen or salinity info - run this cell, otherwise skip!!!**

In [ ]:
# Now process the salts and oxygen data
    # Clean the salinity
clean_sal_files(salts_and_o2_path)
    # Process the salinity files
process_sal_files(salts_and_o2_path)
    # Process the oxygen files
process_oxy_files(salts_and_o2_path)

**====================================================================================================================**
Load the salinity and oxygen:

In [ ]:
sal = pd.read_csv(salts_and_o2_path+'SAL_Summary.csv')
sal.drop(columns='Unnamed: 0', inplace=True)
for colname in list(sal.columns.values):
    sal.rename(columns={colname:'Sal ' + colname}, inplace=True)
sal['Sal Sample ID'] = sal['Sal Case'] + sal['Sal Sample ID'].map(str)

In [ ]:
sal.head()

In [ ]:
oxy = pd.read_csv(salts_and_o2_path+'OXY_Summary.csv')
oxy.drop(columns='Unnamed: 0', inplace=True)
for colname in list(oxy.columns.values):
    oxy.rename(columns={colname:'Oxy ' + colname}, inplace=True)
oxy['Oxy Sample ID'] = oxy['Oxy Case'] + oxy['Oxy Sample ID'].map(str)

In [ ]:
oxy.head()

**========================================================================================================================**
### CTD Sampling Log
Load in the CTD sampling log summary sheet. The summary sheet needs to be manually created and the data cleaned before attempting to import. Additionally, ensure that there is only one header line and that it is at the top of the file.

In [ ]:
sample_log = pd.read_excel(log_path,sheet_name='Summary',header=0)
sample_log = sample_log.sort_values(by=['Station-Cast #','Niskin #'])
for colname in list(sample_log.columns.values):
    sample_log.rename({colname:'Log ' + colname},axis='columns',inplace=True)
sample_log.head()

In [ ]:
def strip_x(x):
    if type(x) == str:
        x = x.replace('.','')
        return x
    else:
        return x

In [ ]:
sample_log['Log Nitrate Bottle 1'] = sample_log['Log Nitrate Bottle 1'].apply(lambda x: strip_x(x))

In [ ]:
# Reformat the sample_log start date and time as well as the date/time
sample_log['Log Start Date'] = sample_log['Log Start Date'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d'))
sample_log['Log Start Time'] = sample_log['Log Start Time'].apply(lambda x: str(x))
sample_log['Log Start Time'] = sample_log['Log Start Date'] + 'T' + sample_log['Log Start Time'] + 'Z'

In [ ]:
# Date and Time
sample_log['Log Date'] = pd.to_datetime(sample_log['Log Date']).apply(lambda x: x.strftime('%Y-%m-%d') if not pd.isnull(x) else '')
sample_log['Log Time'] = sample_log['Log Time'].apply(lambda x: x.strftime('%H:%M:%S') if not pd.isnull(x) else '')
sample_log['Log Time'] = sample_log['Log Date'] + 'T' + sample_log['Log Time'] + 'Z'

**========================================================================================================================**
### Merge the CTD-Bottle Data and Sample Log
The next step is to merge the CTD-Bottle data with the sample log using an outer merge based on the cast and niskin/bottle position. The outer merge means that all data will be retained, so that we do not accidentally discard either data-only casts or casts not recorded on the sample logs.

In [ ]:
#summary = bottles.merge(sample_log, how='outer', right_on=['Station-Cast #','Niskin #'], left_on=['Cast','Bottle Position'])
#summary = bottles.merge(sample_log, how='outer', right_on=['Station-Cast #'], left_on=['Cast'])

In [ ]:
summary = sample_log.merge(bottles, how='outer', left_on=['Log Station-Cast #','Log Niskin #'], right_on=['CTD Cast','CTD Bottle Position'])
summary

Fill in missing data based on the sample log info:

In [ ]:
summary['CTD Start Latitude [degrees]'] = summary['CTD Start Latitude [degrees]'].fillna(value=summary['Log Start Latitude'])
summary['CTD Start Longitude [degrees]'] = summary['CTD Start Longitude [degrees]'].fillna(value=summary['Log Start Longitude'])
summary['CTD Start Time [UTC]'] = summary['CTD Start Time [UTC]'].fillna(value=summary['Log Start Time'])
summary['Log Station-Cast #'] = summary['Log Station-Cast #'].fillna(value=summary['CTD Cast'])
summary['CTD Bottle Position'] = summary['CTD Bottle Position'].fillna(value=summary['Log Niskin #']);
summary['CTD Date Time'] = summary['CTD Date Time'].fillna(value=summary['Log Time'])
summary['CTD Depth [salt water, m]'] = summary['CTD Depth [salt water, m]'].fillna(value=summary['Log Trip Depth'])

Eliminate redundant or non-useful columns from the existing dataframe:

In [ ]:
summary.drop(columns=['Log Start Latitude','Log Start Longitude','Log Start Date','Log Start Time','CTD Cast',
                     'Log Niskin #','Log Rosette Position','Log Date','Log Time','Log Trip Depth'], inplace=True)

**====================================================================================================================**
Now, split rows which have multiple entries into their own individual rows/entries

In [ ]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

**========================================================================================================================**
Merge the discrete salinity and oxygen data into the sample_log based on the cast and niskin number. Do not use the sample bottle number - it is not stored in the processed discrete data we get back from the labs:

In [ ]:
summary = summary.merge(sal, how='left', left_on=['Log Station-Cast #','Log Salts Bottle #'], right_on=['Sal Station','Sal Sample ID'] )
summary['Sal Salinity [psu]'] = summary['Sal Salinity [psu]'].fillna(value=summary['Log Salts Bottle #'])
#summary.rename(columns={'Sal Salinity [psu]': 'Discrete Salinity [psu]'}, inplace=True)

In [ ]:
# Check that the values match up
check = summary['Log Salts Bottle #'] == summary['Sal Sample ID']
if all(check) is False:
    print(summary[check == False][['Log Station-Cast #','Log Salts Bottle #','Sal Station','Sal Sample ID']])

Drop the unnecessary or extraneous columns:

In [ ]:
summary.drop(columns=['Log Salts Bottle #','Sal Cruise','Sal Station','Sal Case','Sal Sample ID'], inplace=True)

Oxygen data:

In [ ]:
summary = summary.merge(oxy, how='left', left_on=['Log Station-Cast #','Log Oxygen Bottle #'], right_on=['Oxy Station','Oxy Sample ID'] )
summary['Oxy Oxygen [mL/L]'] =  summary['Oxy Oxygen [mL/L]'].fillna(value=summary['Log Oxygen Bottle #'])
#summary.rename(columns={'Oxygen [mL/L]':'Discrete Oxygen [mL/L]'}, inplace=True)

In [ ]:
# Check that the values match up
check = summary['Log Oxygen Bottle #'] == summary['Oxy Sample ID']
if all(check) is False:
    print(summary[check == False][['Log Station-Cast #','Log Oxygen Bottle #','Oxy Sample ID']])


In [ ]:
summary.drop(columns=['Log Oxygen Bottle #','Oxy Cruise','Oxy Station','Oxy Case','Oxy Sample ID','Oxy Units',
                     ], inplace=True)

**========================================================================================================================**
### Nutrients Data
Load the nutrients data (if it exists) and merge with the summary sheet. If the nutrients data has not been returned yet, we fill in the relevant columns with the data from the sampling logs.

In [ ]:
def clean_entry(x):
    if type(x) == float:
        return x
    else:
        x = x.replace(' ','')
        return x

In [ ]:
summary['Log Nitrate Bottle 1'] = summary['Log Nitrate Bottle 1'].apply(lambda x: clean_entry(x))

In [ ]:
try:
    nutrients = pd.read_excel(nutrients_path,sheet_name='Summary',header=0)
    nutrients
except IsADirectoryError:
    nutrients = pd.DataFrame(data=sample_log['Log Nitrate Bottle 1'])
    nutrients.rename(columns={'Log Nitrate Bottle 1':'Sample ID'}, inplace=True)
    columns = ['Sample ID','Cruise','Avg: Nitrate + Nitrite [µmol/L]','Avg: Ammonium [µmol/L]',
               'Avg: Phosphate [µmol/L]','Avg: Silicate [µmol/L]','Avg: Nitrite [µmol/L]','Avg: Nitrate [µmol/L]']
    for col in columns:
        if col not in nutrients.columns.values:
            nutrients[col] = nutrients['Sample ID']

In [ ]:
nutrients.head()

In [ ]:
nutrients.rename(columns=lambda x: x.replace('Avg:', 'Nuts'), inplace=True)
nutrients.rename(columns={'Sample ID':'Nuts Sample ID'}, inplace=True)

Now we can merge into the summary sheet:

In [ ]:
summary = summary.merge(nutrients, how='left', left_on=['Log Nitrate Bottle 1'], right_on=['Nuts Sample ID'])

In [ ]:
summary.head()

In [ ]:
summary[['Log Nitrate Bottle 1','Nuts Sample ID']]

In [ ]:
summary.drop(columns=['Log Nitrate Bottle 1','Nuts Sample ID'], inplace=True)

**========================================================================================================================**
### Chlorophyll Data
If the Chlorophyll measurements have not been returned yet, we will generate a synthetic chlorophyll spreadsheet which substitutes the sample bottle numbers in place of the actual measurements. One complication is that the Chlorophyll sample # column title is not identical between cruises.

In [ ]:
try:
    chl = pd.read_excel(chl_path)
    chl.head()
except IsADirectoryError:
    # If there is no chlorophyll sheet yet, need to copy the bottle data into the final sample log
    chl = sample_log[['Log Station-Cast #','Log Chlorophyll Brown Bottle #','Log Chlorophyll Filter Sample #','Log Chlorophyll LN Tube']]
    chl.rename(columns={
        'Log Chlorophyll Brown Bottle #': 'Brown Bottle #',
        'Log Chlorophyll Filter Sample #': 'Chl (ug/l)',
        'Log Chlorophyll LN Tube': 'Phaeo (ug/l)'
    }, inplace=True)

In [ ]:
for colname in list(chl.columns.values):
    chl.rename({colname: 'Chloro ' + colname}, axis='columns', inplace=True)
chl.rename(columns = lambda x: x.replace(':','').replace('\n',''), inplace=True)

Select a subset of the chlorophyll data which we will merge with the summary spreadsheet

In [ ]:
chl.columns

In [ ]:
chl = chl[['Chloro Cruise #','Chloro Station-Cast #','Chloro Niskin #','Chloro Brown Bottle #','Chloro Filter Sample #',
          'Chloro Chl (ug/l)','Chloro Phaeo (ug/l)','Chloro Comments']]

In [ ]:
chl

In [ ]:
chl.info()

In [ ]:
summary.drop(columns=[x for x in list(summary.columns.values) if 'Chloro ' in x], inplace=True)

In [ ]:
summary = summary.merge(chl, how='left', left_on=['Log Station-Cast #','Log Chlorophyll Filter Sample #'], right_on=['Chloro Station-Cast #','Chloro Filter Sample #'])

In [ ]:
check = summary['Log Chlorophyll Filter Sample #'] == summary['Chloro Filter Sample #']
if all(check) is False:
    print(summary[check == False][['Log Chlorophyll Filter Sample #','Chloro Filter Sample #']])

In [ ]:
summary.drop(columns=['Log Chlorophyll Brown Bottle #','Log Chlorophyll Filter Sample #',
                      'Log Chlorophyll Brown Bottle Volume','Log Chlorophyll LN Tube',
                     ], inplace = True)

In [ ]:
#chl.dropna(subset=['Brown Bottle #'], inplace=True)

In [ ]:
#summary = summary.merge(chl, how='outer', left_on=['Station-Cast #','Chlorophyll Brown Bottle #'], right_on=['Station-Cast #','Brown Bottle #'])

In [ ]:
#summary.drop(columns=['Chlorophyll Brown Bottle #','Chlorophyll Filter Sample #','Chlorophyll LN Tube','Brown Bottle #',
#                     'Chlorophyll Brown Bottle Volume'], inplace = True)

**========================================================================================================================**
### Carbon-System Measurements
If the Carbon system measurements have not been returned yet, we will generate a synthetic DIC spreadsheet which substitutes the sample bottle numbers in place of the actual measurements.

In [ ]:
try:
    dic = pd.read_excel(dic_path,header=0)
    dic
except IsADirectoryError:
    dic = sample_log[['Log Station-Cast #','Log Niskin #','Ph Bottle #','DIC/TA Bottle #']]
    dic.rename(columns={
        'Station-Cast #':'CAST_NO',
        'Niskin #':'NISKIN_NO',
        'DIC/TA Bottle #':'DIC_UMOL_KG',
        'Ph Bottle #':'PH_TOT_MEA',
    }, inplace=True)
    columns = ['CAST_NO', 'NISKIN_NO','DIC_UMOL_KG', 'DIC_FLAG_W', 'TA_UMOL_KG',
       'TA_FLAG_W', 'PH_TOT_MEA', 'TMP_PH_DEG_C', 'PH_FLAG_W']
    for col in columns:
        if col not in dic.columns.values:
            if 'dic' in col.lower() or 'ta' in col.lower():
                dic[col] = dic['DIC_UMOL_KG']
            elif 'ph' in col.lower():
                dic[col] = dic['PH_TOT_MEA']
            else:
                dic[col] = np.nan

In [ ]:
dic.columns.values

In [ ]:
#dic = dic[['CAST_NO', 'NISKIN_NO','DIC_UMOL_KG', 'DIC_FLAG_W', 'TA_UMOL_KG',
       #'TA_FLAG_W', 'PH_TOT_MEA', 'TMP_PH_DEG_C', 'PH_FLAG_W']]
#dic.rename(columns = {'DIC_UMOL_KG':'DIC [µmol/kg]','DIC_FLAG_W':'DIC Flag',
#               'TA_UMOL_KG':'Alkalinity [µmol/kg]',
 #              'TA_FLAG_W':'Alkalinity Flag',
  #             'PH_TOT_MEA':'pH [Total Scale]',
   #            'TMP_PH_DEG_C':'pH Analysis Temp [C]', 
    #           'PH_FLAG_W':'pH Flag'}, inplace=True)
# Add in the pCO2 columns, which we don't measure
dic['PCO2_UMOL_KG'] = np.nan
dic['PCO2_FLAG_W'] = np.nan
dic['TMP_PCO2_DEG_C'] = np.nan

dic.rename(columns=lambda x: 'CARBON ' + x, inplace=True)

In [ ]:
dic

In [ ]:
summary = summary.merge(dic, how='left', left_on=['Log Cruise ID','Log Station-Cast #','CTD Bottle Position'], right_on=['CARBON CRUISE_ID','CARBON CAST_NO','CARBON NISKIN_NO'])
#summary = summary.merge(dic, how='left', left_on='DIC/TA Bottle #', right_on='Discrete SAMPLE_ID')

In [ ]:
summary[['Log Cruise ID','Log Station-Cast #','CTD Bottle Position','CARBON CRUISE_ID','CARBON CAST_NO','CARBON NISKIN_NO']]

In [ ]:
summary.drop_duplicates(inplace=True)

**====================================================================================================================**
Next step is to select the desired columns from the total superset of data. We'll do this by setting up a list which contains the key columns that we want for each parameter, and use masking to select those from the superset

In [ ]:
summary.columns.values

In [ ]:
columns = ['Log Cruise ID', 'Log Station-Cast #', 'Log Target Station', 'CTD Start Latitude [degrees]', 
           'CTD Start Longitude [degrees]', 'CTD Start Time [UTC]', 'Log Bottom Depth [m]', 'CTD Filename',
           'CTD Bottle Position','CTD Date Time', 'CTD Pressure, Digiquartz [db]', 'CTD Depth [salt water, m]',
           'CTD Latitude [deg]', 'CTD Longitude [deg]', 'CTD Temperature [ITS-90, deg C]',
           'CTD Temperature, 2 [ITS-90, deg C]', 'CTD Conductivity [S/m]', 'CTD Conductivity, 2 [S/m]', 
           'CTD Salinity, Practical [PSU]', 'CTD Salinity, Practical, 2 [PSU]', 'CTD Oxygen, SBE 43 [ml/l]', 
           'CTD Oxygen Saturation, Garcia & Gordon [ml/l]', 'CTD Beam Attenuation, WET Labs C-Star [1/m]',
           'CTD Beam Transmission, WET Labs C-Star [%]', 'Oxy Oxygen [mL/L]', 'Chloro Chl (ug/l)',
           'Chloro Phaeo (ug/l)', 'Nuts Phosphate [µmol/L]', 'Nuts Silicate [µmol/L]',
           'Nuts Nitrate+Nitrite [µmol/L]',  'Nuts Ammonium [µmol/L]',
           'Sal Salinity [psu]', 'CARBON TA_UMOL_KG', 'CARBON TA_FLAG_W', 'CARBON DIC_UMOL_KG', 'CARBON DIC_FLAG_W',
           'CARBON PCO2_UMOL_KG', 'CARBON TMP_PCO2_DEG_C', 'CARBON PCO2_FLAG_W', 'CARBON PH_TOT_MEA',
           'CARBON TMP_PH_DEG_C', 'CARBON PH_FLAG_W', 'Log Comments', 'Chloro Comments']
   # 'Nuts Nitrite [µmol/L]',       Put this back into place - removed for Pioneer-03 and before

In [ ]:
summary_sheet = summary[[x for x in columns]]
summary_sheet.head()

In [ ]:
summary_sheet.rename(columns={'CTD Date Time':'CTD Bottle Closure'}, inplace=True)
summary_sheet.rename(columns={'Chloro Comments': 'Chl Comments'}, inplace=True)

Now, strip off the source name (i.e. Log/CTD/Sal/etc.) and replace with the appropriate name following the agreed-upon naming convention.

In [ ]:
summary_sheet.rename(columns=lambda x: x.replace('Log ','').replace('CTD ','').replace('Nuts ','Discrete ').replace('Sal ','Discrete ').replace('Oxy ','Discrete ').replace('Chloro ','Discrete '),
                    inplace=True)
summary_sheet.rename(columns=lambda x: x.replace('CARBON ',''), inplace=True)
summary_sheet.rename(columns = {'DIC_UMOL_KG':'Discrete DIC [µmol/kg]','DIC_FLAG_W':'Discrete DIC Flag',
               'TA_UMOL_KG':'Discrete Alkalinity [µmol/kg]',
               'TA_FLAG_W':'Discrete Alkalinity Flag',
               'PH_TOT_MEA':'Discrete pH [Total Scale]',
               'TMP_PH_DEG_C':'Discrete pH Analysis Temp [C]', 
               'PH_FLAG_W':'Discrete pH Flag', 
               'PCO2_UMOL_KG':'Discrete pCO2 [µmol/kg]',
               'TMP_PCO2_DEG_C':'Discrete pCO2 Analysis Temp [C]',
               'PCO2_FLAG_W':'Discrete pCO2 Flag'}, inplace=True)

In [ ]:
summary_sheet.columns

In [ ]:
summary_sheet.sort_values(by=['Cruise ID','Station-Cast #','Bottle Position'], inplace=True)

**========================================================================================================================**
Import the column order list and use fuzzy string matching to sort the data and save the data to an new Excel spreadsheet.

In [ ]:
column_order = pd.read_excel(basepath+'column_order.xlsx')

In [ ]:
column_order = tuple([x.replace('CTD','').strip() for x in column_order.columns.values])

In [ ]:
column_order

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
results = {}
CTDsorted = pd.DataFrame()
for column in column_order:
    match = process.extractBests(column.replace('Discrete ','').replace('Calculated ',''),
                                 summary_sheet.columns.values, limit=2, score_cutoff=56, scorer=fuzz.ratio)
    if 'calculated' in column.lower():
        CTDsorted[column] = -9999999
    elif 'Nitrate' in column:
        CTDsorted['Discrete Nitrate+Nitrite [µmol/L]'] = summary_sheet['Discrete Nitrate+Nitrite [µmol/L]']
    elif 'Nitrite' in column:
        pass
    elif 'flag' in column.lower():
        if column not in ['Discrete DIC Flag','Discrete Alkalinity Flag','Discrete pCO2 Flag','Discrete pH Flag']:
            CTDsorted[column] = -9999999
        else:
            CTDsorted[column] = summary_sheet[column]
            results.update({column:match[0]})
    elif len(match) == 0:
        CTDsorted[column] = -9999999
    elif (match[0][0] not in [x[0] for x in results.values()]):
        CTDsorted[match[0][0]] = summary_sheet[match[0][0]]
        results.update({column:match[0]})
    elif len(match) == 1:
        CTDsorted[match[0][0]] = summary_sheet[match[0][0]]
        results.update({column:match[0]})
    else:
        CTDsorted[match[1][0]] = summary_sheet[match[1][0]]
        results.update({column:match[1]})
CTDsorted['Comments'] = summary_sheet['Comments']
CTDsorted['Chl Comments'] = summary_sheet['Chl Comments']

In [ ]:
CTDsorted.rename(columns = {'Cruise #:':'Cruise ID'}, inplace=True)
CTDsorted.sort_values(by=['Station-Cast #','Bottle Position'], inplace=True)

In [ ]:
cruise_id = list(set(CTDsorted['Cruise ID'].dropna()))
CTDsorted['Cruise ID'] = CTDsorted['Cruise ID'].fillna(value=cruise_id[0])

In [ ]:
cruise_name = cruise.replace('/','').split('_')[0]
current_date = pd.to_datetime(pd.datetime.now()).tz_localize(tz='US/Eastern').tz_convert(tz='UTC')
version = '1-02'

In [ ]:
cruise_id, cruise_name

In [ ]:
filename = '_'.join([cruise_name,cruise_id[0],'Discrete','Summary',current_date.strftime('%Y-%m-%d'),'ver',version,'.xlsx'])
filename

In [ ]:
CTDsorted.fillna(value=-9999999,inplace=True)

In [ ]:
CTDsorted.to_excel(basepath+array+cruise+filename)

**========================================================================================================================**


In [6]:
#summary_file = '/home/andrew/Documents/OOI-CGSN/ooicgsn-water-sampling/Pioneer-07_AR-08_Discrete_Summary_2019-06-25_ver_1-01_.xlsx'
summary_file = basepath+array+cruise+'Pioneer-03_KN-222_Discrete_Summary_2019-07-30_ver_1-02_.xlsx'
summary_file

'/home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Pioneer/Pioneer-03_Leg-1_KN-222_2014-10-03/Pioneer-03_KN-222_Discrete_Summary_2019-07-30_ver_1-02_.xlsx'

In [7]:
summary = pd.read_excel(summary_file)
#summary.drop(columns='Unnamed: 0', inplace=True)
summary

,Cruise ID,Station-Cast #,Target Station,Start Latitude [degrees],Start Longitude [degrees],Start Time [UTC],Cast,Cast Flag,Bottom Depth [m],Filename,...,Calculated DIC [µmol/kg],Calculated pCO2 [µatm],Calculated pH,Calculated CO2aq [µmol/kg],Calculated bicarb [µmol/kg],Calculated CO3 [µmol/kg],Calculated Omega-C,Calculated Omega-A,Comments,Chl Comments
0,KN-222,1,OSPM,39 56.40 N,070 53.00 W,2014-10-04T15:23:26Z,1,100,445,ctd\kn222001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,very low reading on fluorometer
1,KN-222,1,OSPM,39 56.40 N,070 53.00 W,2014-10-04T15:23:26Z,1,100,445,ctd\kn222001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,very low reading on fluorometer
2,KN-222,1,OSPM,39 56.40 N,070 53.00 W,2014-10-04T15:23:26Z,1,100,445,ctd\kn222001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
3,KN-222,1,OSPM,39 56.40 N,070 53.00 W,2014-10-04T15:23:26Z,1,100,445,ctd\kn222001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
4,KN-222,1,OSPM,39 56.40 N,070 53.00 W,2014-10-04T15:23:26Z,1,100,445,ctd\kn222001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
5,KN-222,1,OSPM,39 56.40 N,070 53.00 W,2014-10-04T15:23:26Z,1,100,445,ctd\kn222001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
6,KN-222,1,OSPM,39 56.40 N,070 53.00 W,2014-10-04T15:23:26Z,1,100,445,ctd\kn222001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
7,KN-222,1,OSPM,39 56.40 N,070 53.00 W,2014-10-04T15:23:26Z,1,100,445,ctd\kn222001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
8,KN-222,1,OSPM,39 56.40 N,070 53.00 W,2014-10-04T15:23:26Z,1,100,445,ctd\kn222001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
9,KN-222,1,OSPM,39 56.40 N,070 53.00 W,2014-10-04T15:23:26Z,1,100,445,ctd\kn222001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999


In [8]:
cols = [x for x in summary.columns if 'flag' in x.lower()]

In [9]:
def reformat_numbers(x):
    if x == -9999999:
        return x
    elif x == 9:
        return -9999999
    else:
        x = str(x).zfill(16)
        return x

In [10]:
for col in cols:
    summary[col] = summary[col].apply(lambda x: reformat_numbers(x))

In [11]:
summary['Discrete Alkalinity Flag']

0     0000000000000100
1     0000000000000100
2     0000000000000100
3             -9999999
4     0000000000000100
5     0000000000000100
6     0000000000000100
7             -9999999
8             -9999999
9             -9999999
10            -9999999
11            -9999999
12            -9999999
13            -9999999
14            -9999999
15            -9999999
16            -9999999
17            -9999999
18            -9999999
19            -9999999
20            -9999999
21            -9999999
22            -9999999
23            -9999999
24    0000000000000100
25            -9999999
26    0000000000000100
27    0000000000000100
28    0000000000000100
29            -9999999
            ...       
69            -9999999
70            -9999999
71            -9999999
72            -9999999
73            -9999999
74            -9999999
75    0000000000000100
76            -9999999
77    0000000000000100
78            -9999999
79    0000000000000100
80            -9999999
81    00000

In [12]:
summary.to_csv(basepath+array+cruise+'Pioneer-03_KN-222_Discrete_Summary_2019-07-30_ver_1-02.csv')

In [13]:
filename

NameError: name 'filename' is not defined

In [ ]:
chl = pd.read_excel(chl_path+'Pioneer-03_Leg-1_KN-222_Chlorophyll_Sample_Data_2017-09-21_ver_1-00.xlsx')

In [ ]:
chl

In [ ]:
for ind in summary.index:
    chl_sample = summary['Discrete Chl (ug/l)'].iloc[ind]
    subset = chl[chl['Filter \nSample #'] == chl_sample]
    if len(subset) == 0:
        continue
    else:
        chloro = float(subset['Chl (ug/l)'])
        phaeo = float(subset['Phaeo (ug/l)'])
    summary['Discrete Chl (ug/l)'].iloc[ind] = chloro
    summary['Discrete Phaeo (ug/l)'].iloc[ind] = phaeo

In [ ]:
summary['Discrete Phaeo (ug/l)']

In [ ]:
chl['Filter \nSample #']

In [ ]:
def change_cruise(x):
    if x == 'AT27-A':
        x = 'AT-27A'
    elif x == 'AT27-B':
        x = 'AT-27B'
    else:
        x = x
    return x

In [ ]:
dic['CRUISE_ID'] = dic['CRUISE_ID'].apply(lambda x: change_cruise(x))
dic.columns

In [ ]:
for ind in summary.index:
    cid = summary['Cruise ID'].iloc[ind]
    sta = summary['Station-Cast #'].iloc[ind]
    bot = summary['Bottle Position'].iloc[ind]
    # Get the a subset of the dic 
    subset = dic[(dic['CRUISE_ID'] == cid) & (dic['CAST_NO'] == sta) & (dic['NISKIN_ID'] == bot)]
    # If the subset is empty, continue, else get the unique data
    if len(subset) == 0:
        continue
    else:
        # dic data
        co2 = subset['DIC_UMOL_KG']
        co2flag = subset['DIC_FLAG_W']
        # alkalinity 
        alk = subset['TA_UMOL_KG']
        alkflag = subset['TA_FLAG_W']
        # pH data
        pH = subset['PH_TOT_MEA']
        pHt = subset['TMP_PH_DEG_C']
        pHflag = subset['PH_FLAG_W']
    # Now fill in the relevant data
    summary['Discrete DIC [µmol/kg]'].iloc[ind] = float(co2)
    summary['Discrete DIC Flag'].iloc[ind] = float(co2flag)
    summary['Discrete Alkalinity [µmol/kg]'].iloc[ind] = float(alk)
    summary['Discrete Alkalinity Flag'].iloc[ind] = float(alkflag)
    summary['Discrete pH [Total Scale]'].iloc[ind] = float(pH)
    summary['Discrete pH Analysis Temp [C]'].iloc[ind] = float(pHt)
    summary['Discrete pH Flag'].iloc[ind] = float(pHflag)


In [ ]:
summary['Discrete DIC [µmol/kg]']

In [ ]:
summary.to_csv('/home/andrew/Documents/OOI-CGSN/ooicgsn-water-sampling/Pioneer-02_KN-217_Discrete_Summary_2019-07-11_ver_1-00.csv')

In [ ]:
ctd_log_b = pd.read_excel(water_path+'Pioneer-11_AR-31B_CTD_Sampling_Log.xlsx',sheet_name='Summary')
ctd_log_b

In [ ]:
ctd_log_c = pd.read_excel(water_path+'Pioneer-11_AR-31C_CTD_Sampling_Log.xlsx',sheet_name='Summary')
ctd_log_c['Cruise ID'] = 'AR-31C'

In [ ]:
filt = []
for ind in summary.index:
    cid = summary['Cruise ID'].iloc[ind]
    sta = summary['Station-Cast #'].iloc[ind]
    bot = summary['Bottle Position'].iloc[ind]
    nutnum = ctd_log_b[(ctd_log_b['Cruise ID'] == cid) & (ctd_log_b['Station-Cast #'] == sta) & (ctd_log_b['Niskin #'] == bot)]['Nitrate Bottle 1']
    nutnum = nutnum.to_list()
    if len(nutnum) == 0:
        continue
    else:
        nutnum = str(nutnum[0])
        if nutnum == 'nan':
            continue
        else:
            summary['Discrete Nitrate [µmol/L]'].iloc[ind] = nutnum

In [ ]:
# Repeat for the second ctd_log
filt = []
for ind in summary.index:
    cid = summary['Cruise ID'].iloc[ind]
    sta = summary['Station-Cast #'].iloc[ind]
    bot = summary['Bottle Position'].iloc[ind]
    nutnum = ctd_log_c[(ctd_log_c['Cruise ID'] == cid) & (ctd_log_c['Station-Cast #'] == sta) & (ctd_log_c['Niskin #'] == bot)]['Nitrate Bottle 1']
    nutnum = nutnum.to_list()
    if len(nutnum) == 0:
        continue
    else:
        nutnum = str(nutnum[0])
        if nutnum == 'nan':
            continue
        else:
            summary['Discrete Nitrate [µmol/L]'].iloc[ind] = nutnum

In [ ]:
def replace_cruise(x):
    if x == 'AR31-B':
        return 'AR-31B'
    elif x == 'AR31-C':
        return 'AR-31C'
    else:
        return x

In [ ]:
nutrients['Cruise'] = nutrients['Cruise'].apply(lambda x: replace_cruise(x))

In [ ]:
nutrients['Cruise']

In [ ]:
summary_b = summary.merge(nutrients, how='left', left_on=['Cruise ID','Discrete Nitrate [µmol/L]'], right_on=['Cruise','Sample ID'])

In [ ]:
summary_b['Discrete Nitrate [µmol/L]_y'].dropna()

In [ ]:
summary_b.to_excel(basepath+array+cruise+summary_name)

In [ ]:
nutnum.

In [ ]:
summary.query()

In [ ]:
nutrients_path = water_path+'Pioneer-08_AR-18_Nutrients_Sample_Data_2017-08-18_ver_1-00.xlsx'

In [ ]:
nutrients = pd.read_excel(nutrients_path)

In [ ]:
nutrients

In [ ]:
summary = summary.merge(nutrients, how='left', left_on='Discrete Nitrate [µmol/L]', right_on='Sample ID')

In [ ]:
summary.info()

In [ ]:
# Rename the columns:
summary['Discrete Nitrate [µmol/L]'] = summary['Avg: Nitrate [µmol/L]']
summary['Discrete Nitrite [µmol/L]'] = summary['Avg: Nitrite [µmol/L]']
summary['Discrete Phosphate [µmol/L]'] = summary['Avg: Phosphate [µmol/L]']
summary['Discrete Ammonium [µmol/L]'] = summary['Avg: Ammonium [µmol/L]']
summary['Discrete Silicate [µmol/L]'] = summary['Avg: Silicate [µmol/L]']

In [ ]:
nutrients.columns.values

In [ ]:
summary.drop(columns=nutrients.columns.values, inplace=True)

In [ ]:
summary.info()

In [ ]:
summary.drop_duplicates(inplace=True)

In [ ]:
summary.info()

In [ ]:
cols = [x for x in summary.columns.values if 'flag' in x.lower()]
cols

In [ ]:
summary.fillna(value=-9999999, inplace=True)

In [ ]:
def fill_flags(x):
    
    if x==-9999999:
        return x
    else:
        x = str(x).zfill(16)
        return x

In [ ]:
for c in cols:
    print(c)
    summary[c] = summary[c].apply(lambda x: fill_flags(x))

In [ ]:
summary['Start Time [UTC]'].iloc[286][-20:]

In [ ]:
summary

In [ ]:
def fix_start_time(x):
    if len(x) > 20:
        x = x[-20:]
        return x
    else:
        return x
        

In [ ]:
summary['Start Time [UTC]'] = summary['Start Time [UTC]'].apply(lambda x: fix_start_time(x))

In [ ]:
cruise_name = cruise.split('_')
cruise_name

In [ ]:
summary['Cruise ID'] = summary['Cruise ID'].fillna(value=cruise_id[0])

In [ ]:
cruise
cruise_id = list(set(summary['Cruise ID'].dropna()))[0].split('-')[0]
current_date = pd.to_datetime(pd.datetime.now()).tz_localize(tz='US/Eastern').tz_convert(tz='UTC')
version = '1-01'

In [ ]:
filename = '_'.join([cruise_name,cruise_id,'Discrete','Summary',current_date.strftime('%Y-%m-%d'),'ver',version])
filename = filename+'.csv'
filename

In [ ]:
summary.to_csv(basepath+array+cruise+filename)

In [ ]:
df

In [ ]:
results = {}
CTDsorted = pd.DataFrame()
for column in column_order:
    match = process.extractBests(column.replace('Discrete ','').replace('Calculated ',''),
                                 df.columns.values, limit=2, score_cutoff=56, scorer=fuzz.ratio)
    if 'calculated' in column.lower():
        CTDsorted[column] = -9999999
    elif 'flag' in column.lower():
        if column not in ['Discrete DIC Flag','Discrete Alkalinity Flag','Discrete pCO2 Flag','Discrete pH Flag']:
            CTDsorted[column] = -9999999
        else:
            CTDsorted[column] = df[column]
            results.update({column:match[0]})
    elif len(match) == 0:
        CTDsorted[column] = -9999999
    elif (match[0][0] not in [x[0] for x in results.values()]):
        CTDsorted[match[0][0]] = df[match[0][0]]
        results.update({column:match[0]})
    elif len(match) == 1:
        CTDsorted[match[0][0]] = df[match[0][0]]
        results.update({column:match[0]})
    else:
        CTDsorted[match[1][0]] = df[match[1][0]]
        results.update({column:match[1]})
CTDsorted['Comments'] = df['Comments']

In [ ]:
df

In [ ]:
for i in df.columns.values:
    print(i)

In [ ]:
summary_name_map = {}
for i,key in enumerate(column_order):
    print(key + ': ' + str(i))

In [ ]:
ctd_name_map = {}
for col in df.columns.values:
    ctd_name_map.update({col: ''})
    

In [ ]:
ctd_name_map = {
    'Bottle Position': 'Niskin/Bottle Position',
    'Date Time': 'Bottle Closure Time [UTC]',
    'Pressure, Digiquartz [db]': 'Pressure [db]',
    'Depth [salt water, m]': 'Depth [m]',
    'Latitude [deg]': 'Latitude [deg]',
    'Longitude [deg]': 'Longitude [deg]',
    'Temperature [ITS-90, deg C]': 'Temperature 1 [deg C]',
    'Temperature, 2 [ITS-90, deg C]': 'Temperature 2 [deg C]',
    'Conductivity [S/m]': 'Conductivity 1 [S/m]',
    'Conductivity, 2 [S/m]': 'Conductivity 2 [S/m]',
    'Salinity, Practical [PSU]': 'Salinity 1, uncorrected [psu]',
    'Salinity, Practical, 2 [PSU]': 'Salinity 2, uncorrected [psu]',
    'Oxygen raw, SBE 43 [V]': None,
    'Oxygen, SBE 43 [ml/l]': 'Oxygen, uncorrected [mL/L]',
    'Oxygen Saturation, Garcia & Gordon [ml/l]': 'Oxygen Saturation [mL/L]',
    'Beam Attenuation, WET Labs C-Star [1/m]': 'Beam Attenuation [1/m]',
    'Beam Transmission, WET Labs C-Star [%]': 'Beam Transmission [%]',
    'Filename': 'File',
    'Start Latitude [degrees]': 'Start Latitude [degrees]',
    'Start Longitude [degrees]': 'Start Longitude [degrees]',
    'Cruise': 'Cruise',
    'Start Time [UTC]': 'Start Time [UTC]',
    'Cast': 'Cast'
}

In [ ]:
sample_log_map = {
    'Cruise ID': 'Cruise',
    'Station-Cast #': 'Station',
    'Target Asset': 'Target Asset',
    'Start Latitude': 'Start Latitude [degrees]',
    'Start Longitude': 'Start Longitude [degrees]',
    'Start Date':'',
    'Start Time':'',
    'Bottom Depth [m]': 'Bottom Depth at Start Position [m]',
    'Niskin #': 'Niskin/Bottle Position',
    'Rosette Position': 'Niskin/Bottle Position',
    'Date': '',
    'Time': '',
    'Trip Depth': 'Depth [m]',
    'Oxygen Bottle #': 'Discrete Oxygen [mL/L]',
    'Ph Bottle #': ['Discrete pH [Total Scale]', ],
    'DIC/TA Bottle #': ,
    'Salts Bottle #': ,
    'Nitrate Bottle 1': ,
    
    
}

In [ ]:
column_order

In [ ]:
df2 = pd.DataFrame()
for key in ctd_name_map.keys():
    df2[ctd_name_map.get(key)] =  df[key]

In [ ]:
df2

In [ ]:
df2.to_excel(basepath+array+cruise+water+'Leg1summary.xlsx')

In [ ]:
194*46767648*(0.75/100)